In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：自定义训练表格回归模型进行批处理预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署一个用于批量预测的自定义表格回归模型。

### 数据集

本教程使用的数据集是[Boston房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本嵌入到了TensorFlow中。训练后的模型可以预测房屋的中位价格，单位为1K美元。

### 目标

在本教程中，您将使用 Vertex 客户端库，在 Google 预构建的 Docker 容器中的 Python 脚本中创建一个自定义模型和训练流水线，然后对上传的模型进行批量预测。您也可以使用 `gcloud` 命令行工具或通过 Google Cloud 控制台在线创建自定义模型。

执行的步骤包括：

- 为模型训练创建一个 Vertex 自定义作业。
- 训练 TensorFlow 模型。
- 检索和加载模型工件。
- 查看模型评估。
- 将模型上传为 Vertex `Model` 资源。
- 进行批量预测。

费用

本教程使用谷歌云(GCP)的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用情况生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 *google-cloud-storage* 库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

安装完 Vertex 客户端库和 Google *cloud-storage* 之后，您需要重新启动笔记本内核，以便它能找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###GPU运行时

*如果有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的Google Cloud项目

**无论你的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个帐户时，你会获得$300的免费信用用于你的计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入你的项目ID。然后运行单元格，确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter会将以`!`为前缀的行作为shell命令运行，并会将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分的操作。 以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶进行Vertex训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行现场教程会话，您可能会使用共享的测试账户或项目。为了避免资源创建时用户之间的名称冲突，您需要为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账号

**如果您正在使用谷歌云笔记本**，您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明来通过oAuth验证您的账号。

**否则**，请按照以下步骤进行操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**栏中输入名称，然后点击**创建**。

在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，然后选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是什么笔记本环境，都需要按照以下步骤进行操作。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 软件包上传到一个云存储桶中。Vertex 将从这个软件包中运行代码。在本教程中，Vertex 还会将您作业得到的训练模型保存在同一个存储桶中。然后，您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请在下方设置您的云存储桶的名称。存储桶名称必须在所有 Google Cloud 项目中是全局唯一的，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时才运行以下代码以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：数据集、模型、作业、流水线和终端服务的顶点 API 服务端点。
- `PARENT`：数据集、模型、作业、流水线和终端资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置训练和预测的硬件加速器（例如GPU）。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`来使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用一个GPU容器映像，其中有4个 Nvidia Telsa K80 GPU 分配给每个虚拟机，您需要指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器映像。

*注意*：在 2.3 版本之前的 TF 对于 GPU 支持将无法加载本教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已经修复 —— 它是由服务函数中生成的静态图操作导致的。如果您在自己的自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将为训练和预测设置Docker容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

如需最新列表，请参见[用于训练的预建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

如需最新列表，请参见[用于预测的预先构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置要用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
  - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
  - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：不支持以下用于训练：*

  - `standard`：2个vCPUs
  - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始为波士顿房产创建自己的定制模型和训练了。

## 设置客户端

Vertex客户端库以客户端/服务器模型工作。在您的一侧（Python脚本），您将创建一个客户端，该客户端向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 用于`Model`资源的模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供预测的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

您可以使用容器镜像的两种方法来训练自定义模型：

- **使用 Google Cloud 预构建容器**。 如果您使用预构建容器，还需要指定要安装到容器镜像中的 Python 包。 这个 Python 包包含了用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。 如果您使用自己的容器，容器需要包含用于训练自定义模型的代码。

准备您的定制作业规范

现在您的客户已经准备好了，您的第一步是为您的定制培训工作创建一个作业规范。作业规范将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规范以及数量(单个或分布式)
- `python_package_spec`：要与预构建容器一起安装的Python包规范。

### 准备您的机器规格

现在为您定制的训练任务定义机器规格。这将告诉 Vertex 需要为训练提供哪种类型的机器实例。
  - `machine_type`：要提供的 GCP 实例类型 -- 例如，n1-standard-8。
  - `accelerator_type`：加速器的类型，如果有的话。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则您正在使用 GPU；否则您将使用 CPU。
  - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这告诉 Vertex 每台机器实例用于训练的磁盘类型和大小。

- `boot_disk_type`：SSD 或 Standard 中的一个。SSD 更快，Standard 更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘的大小，以 GB 为单位。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作人员池规范

接下来，您将定义用于自定义训练作业的工作人员池规范。工作人员池规范将包括以下内容：

- `replica_count`：要配置此机器类型的实例数目。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。

- `python_package`：要安装在VM实例上的Python训练包以及要调用的Python模块，以及用于Python模块的命令行参数。

现在让我们深入了解python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的Docker镜像。

- `package_uris`：这是要安装在配置实例上的Python训练包的位置（URIs）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单个Python文件或整个包的zip（存档）。在后一种情况下，作业服务将解压缩（解压缩）内容到Docker镜像中。

- `python_module`：要调用以运行自定义训练作业的Python模块（脚本）。在本例中，您将调用`trainer.task.py`--请注意，没有必要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：保存模型工件的Cloud Storage位置。有两种方法可以告诉训练脚本在哪里保存模型工件：
      - 直接：将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量`DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给训练脚本（设置变量`DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的时期数。
  - `"--steps=" + STEPS`：每个时期的步数（批次）。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分配策略。
     - `"single"`：单设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。
  - `"--param-file=" + PARAM_FILE`：存储特征归一化值的Cloud Storage位置。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

PARAM_FILE = BUCKET_NAME + "/params.txt"

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
        "--param-file=" + PARAM_FILE,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
        "--param-file=" + PARAM_FILE,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

###组装工作规范

现在组装定制工作规范的完整描述：

- `display_name`：您为此定制工作分配的可读名称。
- `job_spec`：定制工作的规范。
    - `worker_pool_specs`：机器VM实例的规范。
    - `base_output_directory`：这告诉服务在何处保存模型工件的Cloud Storage位置（当变量`DIRECT = False`时）。然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将看一下如何为自定义培训工作装配一个 Python 包。当解压缩时，该包包含以下的目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是用于执行自定义培训工作的 Python 脚本。* 注意 *，当我们在工作池规范中引用它时，我们会用一个点 (`trainer.task`) 替换目录斜线，并去除文件后缀 ( `.py` )。

#### 包装配

在接下来的单元格中，您将装配培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我不会详细说明，只是供您查看。总结如下：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF.Keras 内置数据集中加载波士顿住房数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用指定的 epochs 训练模型（`fit()`）。
- 将经过训练的模型保存到指定模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

### 将训练脚本存储在您的云存储桶中

接下来，您可以将训练文件夹打包成一个压缩的 tar 包，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数 `create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规格。

这个辅助函数调用作业客户端服务的 `create_custom_job` 方法，带有以下参数：

- `parent`：指向 `Dataset`、`Model` 和 `Endpoint` 资源的Vertex位置路径。
- `custom_job`：自定义作业的规格。

您将会显示在`response`对象中返回的一些字段，其中最感兴趣的两个是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您保存这个标识符以供后续步骤使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数:

- `name`: 自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，带有以下参数:

- `name`: 自定义作业的 Vertex 完全限定标识符。

如果你还记得，当你调用 `create_custom_job` 方法时，在 `response.name` 字段中获取了自定义作业的 Vertex 完全限定标识符，并将标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过减去`end_time`从`start_time`来计算训练模型所需的实际时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将其保存在您本地的云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载已保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将模型保存的云存储路径传递给它，路径由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来看看模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`加载波士顿房屋测试（留存）数据，使用`load_data()`方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和相应的标签（自住房的中位数价值）。

您不需要训练数据，因此我们将其加载为`(_, _)`。

在您对数据进行评估之前，您需要对其进行预处理：

x_test：
1. 将每一列中的数据进行归一化（重新缩放），即将每个值除以该列的最大值。这将将每个单个值替换为介于0和1之间的32位浮点数。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

现在评估定制作业中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将把您的TF.Keras模型从自定义作业上传到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个serving函数，将数据转换为您的模型所需的格式。如果您将编码数据发送到Vertex，则您的serving函数确保在将数据作为输入传递到模型之前，在模型服务器上解码数据。

### serving函数如何工作

当您向在线预测服务器发送请求时，该请求会被HTTP服务器接收。HTTP服务器从HTTP请求内容体中提取预测请求。提取的预测请求被转发到serving函数。对于Google预构建的预测容器，请求内容被传递给serving函数作为`tf.string`。

serving函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型（动态图）期望的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建JSON对象等。

预处理和后处理函数都被转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建serving函数时，您需要考虑一个需要注意的问题是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果使用，您在编译serving函数时将收到错误提示，指出您正在使用不受支持的EagerTensor。

## 获取服务函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

当进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称 -- 这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

###上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这个步骤，您就可以像处理任何其他Vertex `Model`资源实例一样使用这个`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

这个辅助函数接受以下参数：

- `display_name`：用于`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：我们的SavedModel工件的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存的模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了它。

这个辅助函数调用了`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex `Model`资源实例的规格。

现在让我们更深入地了解Vertex模型规格`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，所以您会将其留空（`''`）。
- `artificat_uri`：保存模型的SavedModel格式的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规格，从这个容器中，`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。您调用response.result()，这是一个同步调用，会在Vertex Model资源准备好时返回。

这个辅助函数返回对应的Vertex Model实例`upload_model_response.model`的Vertex完全限定标识符。您可以将这个标识符保存到变量`model_to_deploy_id`中以备后续步骤使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们获取只有您的模型的模型信息。使用这个辅助函数`get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex `Model`客户端服务的方法`get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 批量预测模型部署

现在部署您为批量预测创建的训练好的Vertex`Model`资源。这与为按需预测部署`Model`资源是不同的。

对于在线预测，您需要：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

3. 向`Endpoint`资源发送在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求提供资源。

3. 将批量预测请求的结果返回给调用者。

4. 作业服务将取消为批量预测请求提供资源。

## 发出一个批量预测请求

现在对您部署的模型进行批量预测。

获取测试项目

您将使用数据集中的测试（保留）部分中的示例作为测试项目。

In [ ]:
test_item_1 = x_test[0]
test_label_1 = y_test[0]
test_item_2 = x_test[1]
test_label_2 = y_test[1]
print(test_item_1.shape)

### 制作批量输入文件

现在制作一个批量输入文件，你将把它存储在本地的云存储桶中。每个预测请求中的实例都是一个字典条目，格式为：

                        {serving_input: content}

- `serving_input`: 底层模型的输入层的名称。
- `content`: 测试项的特征值列表。

In [ ]:
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {serving_input: test_item_1.tolist()}
    f.write(json.dumps(data) + "\n")
    data = {serving_input: test_item_2.tolist()}
    f.write(json.dumps(data) + "\n")

### 计算实例规模

在处理批量预测请求时，您有几种不同的计算实例规模选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动调整规模：批量预测请求在您手动指定的固定数量的计算实例之间分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量都设定为相同。当模型首次部署到实例时，固定数量的计算实例被配置，并且批量预测请求均匀分布在它们之间。

- 自动调整规模：批量预测请求在可以扩展的计算实例数量之间分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时要配置和反配置的实例数量，并根据负载情况将最大（`MAX_NODES`）数量设置为要配置的计算实例数量。

计算实例的最小数量对应字段`min_replica_count`，最大数量对应字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

###进行批量预测请求

现在您的两个测试项的批次已经准备好了，让我们进行批量请求。使用以下帮助函数`create_batch_prediction_job`，并提供以下参数：

- `display_name`：预测作业的人类可读名称。
- `model_name`：`Model`资源的Vertex完全合格标识符。
- `gcs_source_uri`：输入文件的Cloud Storage路径，即您之前创建的文件。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的Cloud Storage路径。
- `parameters`：用于为服务的预测结果添加附加过滤参数。

帮助函数调用作业客户端服务的`create_batch_prediction_job`方法，并提供以下参数：

- `parent`：Dataset、Model和Pipeline资源的Vertex位置根路径。
- `batch_prediction_job`：批量预测作业的规范。

现在让我们深入学习`batch_prediction_job`的规范：

- `display_name`：预测批处理作业的人类可读名称。
- `model`：`Model`资源的Vertex完全合格标识符。
- `dedicated_resources`：为批量预测作业分配的计算资源。
  - `machine_spec`：要分配的计算实例。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。
  - `starting_replica_count`：最初要分配的计算实例数量，您之前设置为变量`MIN_NODES`。
  - `max_replica_count`：要扩展到的计算实例数量的最大值，您之前设置为变量`MAX_NODES`。
- `model_parameters`：用于为服务的预测结果添加附加过滤参数。不支持自定义模型的附加参数。
- `input_config`：要预测的实例的输入来源和格式类型。
 - `instances_format`：批量预测请求文件的格式：`csv`或`jsonl`。
 - `gcs_source`：您批量预测请求的一个或多个Cloud Storage路径列表。
- `output_config`：预测结果的输出目的地和格式。
 - `prediction_format`：批量预测响应文件的格式：`csv`或`jsonl`。
 - `gcs_destination`：预测结果的输出目的地。

此调用是异步操作。您将从响应对象中打印出一些精选字段，包括：

- `name`：分配给批量预测作业的Vertex完全合格标识符。
- `display_name`：预测批处理作业的人类可读名称。
- `model`：Model资源的Vertex完全合格标识符。
- `generate_explanations`：是否提供了与预测相关的True/False解释（可解释性）。
- `state`：预测作业的状态（等待、运行等）。

由于此调用需要一些时间来执行，您可能会得到`JobState.JOB_STATE_PENDING`作为`state`。

In [ ]:
BATCH_MODEL = "boston_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数`get_batch_prediction_job`，并使用以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定标识符。

辅助函数调用作业客户服务的`get_batch_prediction_job`方法，并使用以下参数：

- `name`：批量预测作业的 Vertex 完全限定标识符。在本教程中，您将传递给它您的批量预测作业的 Vertex 完全限定标识符-- `batch_job_id`。

辅助函数将返回存储预测结果的 Cloud 存储路径-- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

获取预测

当批量预测处理完成时，作业状态将是`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在 Cloud Storage 路径中的预测结果。预测结果将以 JSONL 格式呈现，这是在创建批量预测作业时指定的，位于以`prediction`开头的子文件夹下，并在该文件夹中将会有一个名为`prediction.results-xxxxx-of-xxxxx`的文件。

现在显示（cat）内容。您将会看到多个 JSON 对象，每个用于一个预测结果。

响应中包含了每个实例的 JSON 对象，格式如下：

- `dense_input`：用于预测的输入。
- `prediction`：预测的值。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction.results*

        print("Results:")
        ! gsutil cat $folder/prediction.results*

        print("Errors:")
        ! gsutil cat $folder/prediction.errors*
        break
    time.sleep(60)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME